In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
# ls data
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
test = {'key': 'value'}
test['key']
str(test)

"{'key': 'value'}"

In [0]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)
literal_eval(str_dict)[0]
literal_eval(str_dict)[0]['key']
literal_eval(str_dict)[0]['value']
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict
  

df['features_parsed'] = df['features'].map(parse_features)

In [0]:
df['features_parsed'].head()
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [0]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [0]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [0]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_light transmission (vlt)', 'feat_band width',
       'feat_color/finish family', 'feat_designer',
       'feat_date first available at amazon.co.uk', 'feat_pattern',
       'feat_band color', 'feat_country of origin components:', 'feat_year',
       'feat_assembled in country of origin'],
      dtype='object', length=526)

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [0]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
# Transformacja do postaci:

# [
#  {'key': 'Gender', 'value': ['Men']},
#  {'key': 'Shoe Size', 'value': ['M']},
#  {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
#  {'key': 'Color', 'value': ['Multicolor']},
#  {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
#  {'key': 'Brand', 'value': ['Josmo']}]

#  {
#      'Gender': 'Men',
#      'Shoe Size': 'M'...
#  }



In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
   df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [0]:
# df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8846, 531)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.2956608316637, 4.098549795394517)

In [0]:
feats_cat = [x for x in df.columns if '_cat' in x and 'feat_' in x and 'feat_catalog' not in x]
feats_cat

['feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_contains cubic zirconia_cat',
 'feat_age end_cat',
 'feat_variant group id_cat',
 'feat_black_cat',
 'feat_watch band material_cat',
 'feat_tactical_cat',
 'feat_water-resistant_cat',
 'feat_age gender group_cat',
 'feat_lens width_cat',
 'feat_hands_cat',
 'feat_characteristics_cat',
 'feat_walmart no._cat',
 'feat_number of compartments_cat',
 'feat_item spec attr - style (ebay)_cat',
 'feat_authenticity_cat',
 'feat_fabric type_cat',
 'feat_name_cat',
 'feat_autographed_cat',
 'feat_lens color family_cat',
 'feat_number of wheels_cat',
 'feat_sub type_cat',
 'feat_size/dimensions_cat',
 'feat_international shipping_cat',
 'feat_lined_cat',
 'feat_heat zones_cat',
 'feat_outer material_cat',
 'feat_licensed_cat',
 'feat_heel height_cat',
 'feat_condition_cat',
 'feat_nike id_cat',
 'feat_country of origin assembly:_cat',
 '

In [0]:
# feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat',
#          'feat_metal type_cat', 'feat_is weather-resistant_cat', 'feat_leather grade_cat', 'feat_leather :_cat', 'feat_hood_cat', 'feat_shape_cat', 'feat_metal stamp_cat',
#          'feat_fabric_cat', 'feat_is weather-resistant_cat', 'feat_weather-resistant_cat', 'feat_weather resistant_cat',
#          'feat_is waterproof_cat', 'feat_waterproof_cat', 'feat_heel height_cat', 'feat_boot height_cat', 'feat_shoe height_cat',
#          'feat_height_cat', 'feat_made in_cat', 'feat_best sellers rank_cat', 'feat_fabrication_cat', 'feat_age segment_cat', 'feat_occasion_cat']
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat',
		 'feat_shape_cat', 'feat_metal type_cat', 'feat_occasion_cat', 'feat_heel height_cat']
# feats += feats_cat
# feats_full_list = feats
feats = list(set(feats))
feats




['feat_style_cat',
 'feat_shape_cat',
 'feat_occasion_cat',
 'feat_brand_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_metal type_cat',
 'feat_heel height_cat',
 'brand_cat',
 'feat_gender_cat']

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [0]:
X = df[ feats ].values
y = df['prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.19267418240361, 4.252437718468176)


Weight,Feature
0.2562 ± 0.0103,brand_cat
0.0993 ± 0.0108,feat_material_cat
0.0245 ± 0.0038,feat_gender_cat
0.0183 ± 0.0008,feat_brand_cat
0.0125 ± 0.0009,feat_shape_cat
0.0082 ± 0.0011,feat_metal type_cat
0.0030 ± 0.0006,feat_occasion_cat
0.0030 ± 0.0006,feat_style_cat
0.0008 ± 0.0001,feat_heel height_cat
0.0002 ± 0.0000,feat_sport_cat


In [0]:
# df['brand'].value_counts(normalize=True)
# df[ df['brand'] == 'nike'].features_parsed.head().values
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'gender': 'men', 'shoe category': 'mens shoes', 'color': 'wolf grey/team orange/atomic mango/cool grey', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '630913009', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'sport': 'soccer', 'condition': 'new with box'},
       {'sport': 'baseball & softball', 'condition': 'new with box', 'type': 'cleats'},
       {'style': 'athletic sneakers', 'condition': 'new with box', 'country//region of manufacture': 'indonesia', 'pattern': 'solid', 'material': 'synthetic'},
       {'sport': 'football', 'type': 'cleats', 'condition': 'new without box'}],
      dtype=object)

In [0]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
mens                   4
infant                 4
boys'                  3
men||women             2
women ,�� unisex       2
youth                  2
women                  2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64

In [0]:
!git add day5.ipynb
!git config --global user.email "mularczw@o2.pl"
!git config --global user.name "Wojtek"
!git commit -m "Add dw matrix day5 file"
!git push -u origin master